In [1]:
from training_datasets.GLUEMetaDataset import GLUEMetaDataset
from samplers.FewShotEpisodeSampler import FewShotEpisodeSampler
from samplers.FewShotEpisodeBatchSampler import FewShotEpisodeBatchSampler
import torch.utils.data as data

meta_ds = GLUEMetaDataset(k=8, numTasks=2, length=-1)

/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 587.90it/s]
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|█████████████████████████████████████████████| 5/5 [00:00<00:00, 37.73it/s]
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 460.56it/s]
Found cached dataset 

In [2]:
train_protomaml_sampler = FewShotEpisodeBatchSampler(meta_ds, kShot=4, batchSize=2)

train_protomaml_loader = data.DataLoader(
    meta_ds, batch_sampler=train_protomaml_sampler, collate_fn=train_protomaml_sampler.getCollateFunction(), num_workers=4)

# for i in range(1):
#     batch = next(iter(train_protomaml_loader))
#     for episode_i in range(len(batch[0])):
#         data, labels = batch[0][episode_i], batch[1][episode_i]
#         supportSet, supportLabels = data[0:len(data)//2], labels[0:len(data)//2] 
#         print(supportLabels)

In [3]:
# episodes = []
# for i in range(1):
#     episodes.append(gm_ds.getTask())
# for i in range(len(episodes)):
#     classes = len(set(episodes[i][1].tolist()))
#     # print(classes)
#     print(episodes[i][0], episodes[i][1], "\n")

In [4]:
# from training.models.ProtoFOMAML import ProtoFOMAML

# pfomaml = ProtoFOMAML(outerLR=5e-4, innerLR=1e-3, outputLR=1e-2, steps=5, batchSize=16, warmupSteps=0)
# pfomaml.training_step(next(iter(train_protomaml_loader)), 0)

In [5]:
from samplers.FewShotValidationEpisodeBatchSampler import FewShotValidationEpisodeBatchSampler
from samplers.FewShotValidationEpisodeSampler import FewShotValidationEpisodeSampler
from validation_datasets.ValidationDataset import ValidationDataset
import torch.utils.data as data

ds = ValidationDataset()

val_protomaml_sampler = FewShotValidationEpisodeBatchSampler(ds, kShot=8)
val_protomaml_loader = data.DataLoader(
    ds, batch_sampler=val_protomaml_sampler, collate_fn=val_protomaml_sampler.getCollateFunction(), num_workers=1
)

for i in range(1):
    batch = next(iter(val_protomaml_loader))
    for episode_i in range(len(batch[0])):
        data, labels = batch[0][episode_i], batch[1][episode_i]
        supportSet, supportLabels = data[0:len(data)//2], labels[0:len(data)//2] 
#         print(supportSet)
#         print(supportLabels)
#         print("\n")

In [6]:
from training.models.ProtoFOMAML import ProtoFOMAML
from training.models.FOMAML import FOMAML
from training.models.ProtoNet import ProtoNet
from training.models.Reptile import Reptile

# pfomaml = ProtoFOMAML(outerLR=5e-4, innerLR=1e-3, outputLR=1e-2, steps=5, batchSize=16, warmupSteps=0)
# pfomaml.validation_step(next(iter(val_protomaml_loader)), 0)

2023-08-03 11:22:58.816879: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-03 11:22:58.829669: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-03 11:22:58.894427: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-03 11:22:58.894988: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-03 11:22:59.595042: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [ ]:
from training.trainer import train_model
import torch.utils.data as data

protomaml_model = train_model(
    FOMAML,
    train_loader=train_protomaml_loader,
    val_loader=val_protomaml_loader,
    outerLR=5e-5, innerLR=1e-3, outputLR=1e-3, steps=5, batchSize=4, warmupSteps=100
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Using model /home/aksingh/Downloads/version_8230128/checkpoints/working.ckpt


Global seed set to 42

  | Name        | Type               | Params
---------------------------------------------------
0 | metaLearner | PrototypeMetaModel | 108 M 
---------------------------------------------------
14.2 M    Trainable params
94.1 M    Non-trainable params
108 M     Total params
433.241   Total estimated model params size (MB)


Training FOMAML with parameters "batchSize":   4
"innerLR":     0.001
"outerLR":     5e-05
"outputLR":    0.001
"steps":       5
"warmupSteps": 100


/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Number of labels in the episode are 2 and lines are 1
inner loop training loss is 0.7114419937133789 and accuracy is 0.5
inner loop training loss is 0.7091721296310425 and accuracy is 0.5
inner loop training loss is 0.7069917321205139 and accuracy is 0.5
inner loop training loss is 0.7048956155776978 and accuracy is 0.5
inner loop training loss is 0.7028789520263672 and accuracy is 0.5
outer loop episodic validation accuracy is 0.625 and loss is 0.6662566661834717
Number of labels in the episode are 3 and lines are 1
inner loop training loss is 1.0620187520980835 and accuracy is 0.5
inner loop training loss is 1.0596054792404175 and accuracy is 0.5
inner loop training loss is 1.0572313070297241 and accuracy is 0.5
inner loop training loss is 1.0548949241638184 and accuracy is 0.5
inner loop training loss is 1.0525954961776733 and accuracy is 0.5
outer loop episodic validation accuracy is 0.5416666666666666 and loss is 1.0790377855300903
Number of labels in the episode are 3 and lines a

In [ ]:
# from training_datasets.GLUEMetaDataset import GLUEMetaDataset
# from samplers.FewShotEpisodeSampler import FewShotEpisodeSampler
# from samplers.FewShotEpisodeBatchSampler import FewShotEpisodeBatchSampler
# import torch.utils.data as data

# meta_ds = GLUEMetaDataset(k=4,numTasks=1000)

In [ ]:
# from training.models.ProtoNet import ProtoNet
# import torch.utils.data as data
# from training_datasets.GLUEMetaDataset import GLUEMetaDataset
# from training.trainer import train_model

# from samplers.FewShotValidationEpisodeBatchSampler import FewShotValidationEpisodeBatchSampler
# from samplers.FewShotValidationEpisodeSampler import FewShotValidationEpisodeSampler
# from validation_datasets.ValidationDataset import ValidationDataset
# from validation_datasets.GLUEMetaValidationDataset import GLUEMetaValidationDataset
# import torch.utils.data as data

# ds = ValidationDataset()

# val_protomaml_sampler = FewShotValidationEpisodeBatchSampler(ds, kShot=4)
# val_protomaml_loader = data.DataLoader(
#     ds, batch_sampler=val_protomaml_sampler, collate_fn=val_protomaml_sampler.getCollateFunction(), num_workers=1
# )

# validation_dataset = GLUEMetaValidationDataset(k=8, numTasks=5000)
# val_protomaml_loader = data.DataLoader(validation_dataset, num_workers=2)

# train_protonet_loader = data.DataLoader(
#     meta_ds, num_workers=4)

# protomaml_model = train_model(
#     ProtoNet,
#     train_loader=train_protonet_loader,
#     val_loader=val_protomaml_loader,
#     metaLearningRate=5e-5, prototypeLearningRate=1e-2, steps=5, batchSize=8, warmupSteps=0
# )

In [ ]:
from prototypes.models.PrototypeMetaModel import PrototypeMetaModel
import torch

model = PrototypeMetaModel()
model_1 = PrototypeMetaModel()

for param, param_1 in zip(model.parameters(), model_1.parameters()):
    if param.requires_grad:
        if param.grad is None:
            param.grad = torch.zeros(param_1.data.shape)
        param.grad.data += (param.data - param_1.data)
        print(param_1.data)
        print(param.data)
        print(param.grad)

#### 